In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time
import random

########################
# Stocks and Configuration
########################
stocks = [
    'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
    'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
]

train_ratio = 0.9

if not os.path.exists("hyperparam_search_results"):
    os.makedirs("hyperparam_search_results")

# Parameter sets for random sampling
n_steps_options = [5, 10, 15]
n_units_options = [16, 32, 64]
batch_size_options = [2, 4, 8]
initial_lr_options = [0.0005, 0.0002, 0.001]
update_lr = 0.0002
initial_epochs = 100
update_epochs = 100
window_size = 200
step_size = 10
start_date = '2020-01-01'
end_date = '2024-01-01'

def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates

def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def create_lstm_model(n_layers, n_units, learning_rate, input_shape):
    model = Sequential()
    model.add(LSTM(n_units, activation='relu', return_sequences=(n_layers > 1), input_shape=input_shape))
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)
    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None,
                    initial_epochs=100, update_epochs=100,
                    initial_lr=0.0005, update_lr=0.0002,
                    n_units=16, batch_size=4):
    total_samples = len(X_samples)
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(1, n_units, lr, input_shape)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, start_date, end_date,
                                    n_steps=5, window_size=200, step_size=10,
                                    initial_epochs=100, update_epochs=100,
                                    initial_lr=0.0005, update_lr=0.0002,
                                    n_units=16, batch_size=4):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        # Not enough data
        return None

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        # Initial window not valid
        return None
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                                      initial_training=True, prev_weights=None,
                                      initial_epochs=initial_epochs, update_epochs=update_epochs,
                                      initial_lr=initial_lr, update_lr=update_lr,
                                      n_units=n_units, batch_size=batch_size)

    weights_path = f"rolling_error_analysis_results/{ticker}_initial.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                                          initial_training=False, prev_weights=weights_path,
                                          initial_epochs=initial_epochs, update_epochs=update_epochs,
                                          initial_lr=initial_lr, update_lr=update_lr,
                                          n_units=n_units, batch_size=batch_size)

        weights_path = f"rolling_error_analysis_results/{ticker}_iteration_{iteration}.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])
        iteration += 1

    if len(predictions) == 0:
        # No iterations or no predictions made
        return None

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    overall_mape = df_results['MAPE'].mean()
    return overall_mape

if __name__ == "__main__":
    # Parameter sets
    n_steps_options = [5, 10, 15]
    n_units_options = [16, 32, 64]
    batch_size_options = [2, 4, 8]
    initial_lr_options = [0.0005, 0.0002, 0.001]

    # Generate 15 random configurations
    configs = []
    for _ in range(15):
        ns = random.choice(n_steps_options)
        units = random.choice(n_units_options)
        bs = random.choice(batch_size_options)
        ilr = random.choice(initial_lr_options)
        configs.append((ns, units, bs, ilr))

    result_df = pd.DataFrame(columns=['Config','Avg_MAPE'] + stocks)

    global_start = time.time()  # Track total elapsed time for the entire search

    for i, (ns, units, bs, ilr) in enumerate(configs):
        config_start = time.time()
        print(f"\n[INFO] Running config {i+1}/15: n_steps={ns}, n_units={units}, batch_size={bs}, initial_lr={ilr}")
        per_stock_mape = []
        for idx, stock in enumerate(stocks):
            elapsed_since_start = time.time() - global_start
            print(f"[INFO] Starting {stock} (config {i+1}/15), elapsed time: {elapsed_since_start:.2f}s")
            mape = rolling_training_early_stopping(
                stock,
                start_date=start_date,
                end_date=end_date,
                n_steps=ns,
                window_size=window_size,
                step_size=step_size,
                initial_epochs=initial_epochs,
                update_epochs=update_epochs,
                initial_lr=ilr,
                update_lr=0.0002,
                n_units=units,
                batch_size=bs
            )
            if mape is not None:
                per_stock_mape.append(mape)
            else:
                per_stock_mape.append(np.nan)

        avg_mape = np.nanmean(per_stock_mape)
        config_name = f"n_steps={ns},n_units={units},batch_size={bs},initial_lr={ilr}"
        row_data = {'Config': config_name, 'Avg_MAPE': avg_mape}
        for s, m in zip(stocks, per_stock_mape):
            row_data[s] = m
        result_df = result_df.append(row_data, ignore_index=True)

        config_elapsed = time.time() - config_start
        print(f"[INFO] Finished config {i+1}/15, avg MAPE={avg_mape:.2f}%, config time: {config_elapsed:.2f}s, total elapsed: {time.time()-global_start:.2f}s")

    result_df.to_csv("hyperparam_search_results/hyperparam_search_random_15.csv", index=False)
    print("[INFO] Hyperparameter search completed. Results saved to hyperparam_search_results/hyperparam_search_random_15.csv")



[INFO] Running config 1/15: n_steps=10, n_units=32, batch_size=4, initial_lr=0.0002
[INFO] Starting COST (config 1/15), elapsed time: 0.00s
[*********************100%%**********************]  1 of 1 completed


/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `in

/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variab

/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variab

...

/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`in

/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variab

[INFO] Starting UPS (config 3/15), elapsed time: 18635.26s
[*********************100%%**********************]  1 of 1 completed


/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `in

/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Applications/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variab